In [159]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")


rank=1


KT_vars = ['x','y']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)

x,y=KT_ring.gens()
t,m = coho_ring.gens()
b=var('b')
w=SR.var('w',rank)


def character(p,x=x,y=y):
    return sum(y**i * x**j for i, j in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def tangent1(p,x=x):
    q=character(p,x,1/x)
    q_inv=character(p,1/x,x)
    return q+q_inv-q*q_inv*(1-x)*(1-1/x)

def tangent2(p,x=x,y=y):
    total=0
    for b1,b2 in p.cells():
        arm=p[b1]-b2-1
        leg=p.conjugate()[b2]-b1-1
        total+=x^(-leg)*y^(arm+1)+x^(leg+1)*y^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)

def canonical_bundle_red(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)-1

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def total_chern(f):
    numer, denom = from_character(f)
    new_numer=[i+1 for i in numer]
    new_denom=[i+1 for i in denom]
    return coho_ring.fraction_field()(prod(new_numer+[1]) / (prod(new_denom)))

def chern_class(k, f):
    roots = sum((c*[from_monomial(w)] for c, w in f), [])
    return sum(prod(S) for S in itertools.combinations(roots, k))

def Nek(f, check=False):
    R = f.parent()
    numer, denom = R.one(), R.one()
    for exp, coeff in f.dict().items():
        if exp.is_constant():
            term = R.zero()
        else:
            term = R({exp: 1, exp.emul(0): -1})
        if coeff > 0:
            numer *= term ** (coeff)
        elif coeff < 0:
            denom *= term ** (-coeff)
    return R.fraction_field()(numer) / denom

def dual(f, check=False):
    R = f.parent()
    g=R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def highest_degree_y(f):
    R = f.parent()
    max_deg=0
    for exp, coeff in f.dict().items():
        if exp[1]>max_deg:
            max_deg = exp[1]
    term=R.zero()
    for exp, coeff in f.dict().items():
        if exp[1]==max_deg:
            term+=R({exp:coeff})
    return term

def lowest_degree_y(f):
    R = f.parent()
    min_deg=0
    for exp, coeff in f.dict().items():
        if exp[4]<min_deg:
            min_deg = exp[4]
    term=R.zero()
    for exp, coeff in f.dict().items():
        if exp[4]==min_deg:
            term+=R({exp:coeff})
    return term
    
def from_monomial(m):
    R = m.parent()
    if R== KT_ring:
        return m.degree(x) * t 

def top_chern_vb(p):
    Q = character(p,x,y=1/x)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        roots += (coeff) * [wt+sum(w[i]*t for i in range(rank))]
    total = prod(r for r in roots)
    return total

def total_segre_vb(p):
    Q = character(p,x,y=1/x)
    roots = []
    for coeff, mon in list(Q):
        wt = from_monomial(mon)
        roots += (coeff) * [wt+sum(w[i]*t for i in range(rank))]
    total = prod(1/(1+r) for r in roots)
    return total
    
def determinant_vb(p,n):
    Q = character(p,x,y=1/x)
    det = (determinant(Q)^rank)*((SR(x)^(sum(w[i] for i in range(rank))))^n)
    return det(x=exp(t*b))
    
def Chern_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent1(p))*top_chern_vb(p)*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total

def Segre_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent1(p))*total_segre_vb(p)*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total

def Verlinde_Num(n):
    total=0
    for p in Partitions(n):
        add=(KTheory.measure_unsymmetrized(tangent1(p))(x=exp(t*b))*\
            determinant_vb(p,n)*\
            KTheory.measure_unsymmetrized(-canonical_bundle_red(p,x,1/x),inv=False)(x=exp(t*b))).taylor(b,0,4)
        total+=add
    return total

def DT_Num(n):
    total=0
    for p in Partitions(n):
        add=(top_chern(-tangent1(p))*top_chern(dual(canonical_bundle_red(p,x,1/x))))(t=t*b)
        total+=add
    return total

def Verlinde_Num_inv(n):
    total=0
    for p in Partitions(n):
        add=(KTheory.measure_unsymmetrized(tangent1(p))(x=exp(t*b))*\
            (1/determinant_vb(p,n))*\
            KTheory.measure_unsymmetrized(-canonical_bundle_red(p,x,1/x),inv=False)(x=exp(t*b))).taylor(b,0,4)
        total+=add
    return total
        

def degn(f,n):
    g=f.taylor(b,0,n)
    L=g.coefficients()
    if L[len(L)-1][1]!=n:
        return 0
    return L[len(L)-1][0]

We try to get the Verlinde-Segre correspondence for $\mathbb{C}^2$.

Over $\mathbb{C}^2$, the equivariant cohomology ring is $\mathbb{Z}[t_1,t_2]/(t_1+t_2)=\mathbb{Z}[t]$. The equivariant K-theory ring is $\mathbb{Z}[x^{\pm1}]$. The Zariski tangent space at $Z_\pi$ is
$$Q+\bar Q-Q\bar Q(1-x)(1-1/x) $$
where $Q$ is the character of partition $\pi$.

The virtual Segre number is 
$$\int_{[S^{[n]}]^{\text{vir}}} s(\alpha^{[n]})=\sum_{|\pi|=n}e(-T|_Z)s(\alpha^{[n]}|_Z)c_{\text{top}}(K_S^{[n]}|_Z) $$

The Verlinde number is 
$$V_n(\alpha)= \chi(\det (\alpha^{[n]}))=\int_{[S^{[n]}]^{\text{vir}}}\text{td}(T^{\text{vir}})\text{ch}\left( \det (\alpha^{[n]})\right)$$
$$=\sum_{|\pi|=n}e(-T^{\text{vir}}|_Z)\text{td}(T^{\text{vir}}|_Z)\text{ch}\left( \det (\alpha^{[n]})|_Z\right)$$
$$=\sum_{|\pi|=n}\frac{\text{ch}(\wedge K^{[n]}_S|_Z)}{\text{ch}(\wedge T^\vee|_Z)}\text{ch}\left( \det (\alpha^{[n]})|_Z\right) $$
where ${\text{ch}(\wedge x^\vee)}=1/(1-e^{-x})$

Since on $\mathbb{C}^2$, $K_S$ is trivial with no weight from the 1-d torus action, we know $K^{[n]}$ has non-trivial fixed parts, so the above invariaints are 0 (in degree 0 of $H^*_{\mathbb{C}^*}(\text{pt})$). Instead we take the "reduced" obstruction bundle:
$$ K^{[n]}_S-\mathcal{O}_{S^{[n]}}$$
Now we have in degree 0 they agree for $n=1,2,3$. When $n>3$, the fixed part of $K^{[n]}$ has dimension at least 2 at some boxes, but the equality continue to hold.

In [170]:
def segre(w,n):
    return sum(top_chern(-tangent1(p))*total_chern(-character_vb(p,w,x,1/x))*top_chern(dual(canonical_bundle_red(p,x,1/x))) for p in Partitions(n))

In [171]:
W=(7,0)
N=2

tc= segre(W,N)(t=b*t,m=b*(m))
taylc = tc.taylor(b,0,0)
taylc

-21/2/(b*t) + 1/2/(b^2*t^2) + 295/2

In [172]:
def Verlinde(w,n):
    return sum(KTheory.measure_unsymmetrized(tangent1(p),inv=True)* 
               determinant(character_vb(p, w,x,1/x))*KTheory.measure_unsymmetrized(-canonical_bundle_red(p,x,1/x),inv=False)
               for p in Partitions(n))


In [173]:
verl = Verlinde(W,N)(x=exp(t*b),y=exp(m*b))
tayl = verl.taylor(b,0,0)
tayl

-21/2/(b*t) - 3/4/(b^2*t^2) - 147/2

The degree $-1$ term for Segre is $-(w\ /\ nt)$:

In [70]:
pretty(degn(Chern_Num(2),-1))

-1/2*(w0 + w1)/t

The Verlinde number is $(-1)^{n+1}(w\ /\ nt)$:

In [74]:
pretty(degn(Verlinde_Num_inv(2),-1))

-7/2*(w0 + w1 + w2 + w3)/t

and we indeed have correspondence when $\text{rank}=\pm 1$.
When $\text{rank}>1$, Verlinde number at degree $-1$ changes by a multiple of:
$$V_n=\frac{\prod_{i=1}^{r-1}(rn-i)}{(r-1)!} $$

Back to rank $1$, then the segre number $1/(1+x)$ is an infinite power series that has terms in other degrees.

In degree $-2$, the difference (quotient) is
$$\frac{V_n}{S_n}=(-1)^{n+1} \frac{(2n-1)}{n}$$

In degree $0$, the invariants are always polynomials in $w^2$, 
$$[w^2]S_n=(-1)^{n+1}\cdot2\cdot [w^2]V_{n+1} $$
$$\frac{[w^2]S_n}{[w^0]S_n} = \frac{[w^2]V_{n+1}}{[w^0]V_{n+1}}, \text{ for $n\geq 2$ (otherwise division by 0)} $$

Also, we have a recursive formula
$$\frac{[w^2]S_n}{[w^0]S_n}\cdot \frac{(n-1)}{n} = \frac{[w^2]S_{n+1}}{[w^0]S_{n+1}}, n\geq 2$$

In degree $1$, 
$$S_n=(-1)^nV_{n+1}$$

But is it interesting to only look at rank 1 for Verlinde number?

In [115]:
pretty(degn(Segre_Num(6),0))

462*w0^2 + 385

In [97]:
pretty(degn(Verlinde_Num(5),-2))

-1001/25/t^2

In [116]:
pretty(degn(Segre_Num(2),0)-degn(Verlinde_Num(2),0))

9/2*w0^2 + 1/2

# DT inv

We have
$$\int_{[S^{[n]}]}1=(-1)^n\frac{1}{n!t^{2n}}$$
$$\int_{[S^{[n]}]^{\text{red}}}1=\begin{cases}-\frac1{t^2}\text{ when $n=1$}\\0\text{ else}\end{cases}$$

In [160]:
DT_Num(8)

0

In [4]:
str(hex(100))

'0x64'